# AUTOMATIC DIAGNOSTIC SYSTEM OF SKIN LESSIONS FROM DERMOSCOPIC IMAGES


In this practice we are going to build a skin lesion diagnosis system based on dermoscopic image analysis.

## Part 0: The problem

Before starting the practice, we will briefly describe the database that we will use and the problem we aim to address:

Our goal is to develop a CNN providing an automatic diagnosis of cutaneous diseases from dermoscopic images. Dermoscopy is a non-invasive technique that allows the evaluation of the colors and microstructures of the epidermis, the dermoepidermal joint and the papillary dermis that are not visible to the naked eye. These structures are specifically correlated with histological properties of the lesions. Identifying specific visual patterns related to color distribution or dermoscopic structures can help dermatologists decide the malignancy of a pigmented lesion. The use of this technique provides a great help to the experts to support their diagnosis. However, the complexity of its analysis limits its application to experienced clinicians or dermatologists.

In our scenario, we will consider 3 classes of skin lesions:

- Malignant melanoma: Melanoma, also known as malignant melanoma, is the most common type of cancer, and arises from pigmented cells known as melanocytes. Melanomas typically occur on the skin and rarely elsewhere such as the mouth, intestines, or eye.

- Seborrheic keratosis: it is a noncancerous (benign) tumor of the skin that originates from the cells of the outer layer of the skin (keranocytes), so it is a non-melanocytic lesion.

- Benign nevus: a benign skin tumor caused by melanocytes (it is melanocytic)

Figure 1 shows a visual example of the 3 considered lesions:

![Image of ISIC](http://www.tsc.uc3m.es/~igonzalez/images/ISIC.jpg)

The dataset has been obtained from the 'Internatial Skin Imaging Collaboration' (ISIC) file. It contains 2750 images divided into 3 sets:
- Training set: 2000 images
- Validation set: 150 images
- Test set: 600 images

For each clinical case, two images are available:
- The dermoscopic image of the lesion (in the ‘images’ folder).
- A binary mask with the segmentation between injury (mole) and skin (in the 'masks' folder)

Additionally, there is a csv file for each dataset (training, validation and test) in which each lines corresponds with a clinical case, defined with two fields separated by commas:
- the numerical id of the lesion: that allows to build the paths to the image and mask.
- the lesion label: available only for training and validation, being an integer between 0 and 2: 0: benign nevus, 1: malignant melanoma, 2: seborrheic keratosis. In the case of the test set, labels are not available (their value is -1).

Students will be able to use the training and validation sets to build their solutions and finally provide the scores associated with the test set. This practice provides a guideliness to build a baseline reference system. To do so, we will learn two fundamental procedures:

- 1) Process your own database with pytorch
- 2) Fine-tuning a regular network for our diagnostic problem

## Part 1: Handling our custom dataset with pytorch
Now we are going to study how we can load and process our custom dataset in pytorch. For that end, we are going to use the package ``scikit-image`` for reading images, and the package ``panda`` for reading csv files.


In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import pdb
from google.colab import files

from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) # not optimal cpu must be chosen GPU

cuda:0


Lo primero que vamos a hacer es descargar y descomprimir la base de datos a un directorio local de trabajo:




In [2]:
#ONLY TO USE GOOGLE COLAB. Run this code only the first time you run this notebook and then comment these lines
from shutil import copyfile
from google.colab import drive
import os, sys

drive.mount('/content/drive')
copyfile('/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/db1.zip', './db1.zip') #Copy db files to our working folder
copyfile('/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/db2.zip', './db2.zip')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'./db2.zip'

In [0]:
#NOTE: Run this only once, in the machine where you want to run your code, then comment these lines
import zipfile
zipPath='./db1.zip' #path of the 1st zip file
dataFolder='./data' #We extract files to the current folder
with zipfile.ZipFile(zipPath, 'r') as zip_ref:
    zip_ref.extractall(dataFolder)
    
zipPath='./db2.zip' #path of the 2nd zip file
dataFolder='./data' # We extract files to the current folder
with zipfile.ZipFile(zipPath, 'r') as zip_ref:
    zip_ref.extractall(dataFolder)

Now let's read the indexed file and display data from image 65. The file structure is one row per image of the database, and two fields:
- Image ID (a 4-digit sequence, adding 0 to the left side if required)
- Label: 0 benign nevus, 1 melanoma, 2 seborrheic keratosis



In [0]:
#db = pd.read_csv('data/dermoscopyDBtrain.csv',header=0,dtype={'id': str, 'label': int})

#We show inform
#n = 65
#img_id = db.id[65] 
#label = db.label[65]


#print('Image ID: {}'.format(img_id))
#print('Label: {}'.format(label))


Now, let's create a simple function to show an image.




In [0]:
def imshow(image, title_str):
    if len(image.shape)>2:
        plt.imshow(image)
    else:
        plt.imshow(image,cmap=plt.cm.gray)
    plt.title(title_str)        

#plt.figure()
#imshow(io.imread(os.path.join('data/images/', img_id + '.jpg' )),'Image %d'%n)
#plt.figure()
#imshow(io.imread(os.path.join('data/masks/', img_id + '.png')),'Mask %d'%n)

#plt.show()

### Class Dataset

The class `` torch.utils.data.Dataset`` is an abstract class that represents a dataset.

To create our custom dataset in pytorch we must inherit from this class and overwrite the following methods:

- `` __len__`` so that `` len (dataset) `` returns the size of the dataset.
- `` __getitem__`` to support indexing `` dataset [i] `` when referring to sample $i$

We are going to create the train and test datasets of our diagnostic problem. We will read the csv in the initialization method `` __init__`` but we will leave the explicit reading of the images for the method
`` __getitem__``. This approach is more efficient in memory because all the images are not loaded in memory at first, but are read individually when necessary.

Our dataset is going to be a dictionary `` {'image': image, 'mask': mask, 'label': label} ``. You can also take an optional `` transform '' argument so that we can add pre-processing and data augmentation techniques.



In [0]:
def load_data (data_dir, dataset): 
    
#   dataset = pd.read_csv(csv_file,header=0, dtype={'id': str, 'label': int})
    images = []
    masks = []
    labels = []
    for i,idx in enumerate(dataset.id):
        #Leemos la imagen
        img_name = os.path.join(data_dir+'/images', idx + '.jpg')
        image = io.imread(img_name)
        #Leemos la máscara
        mask_name = os.path.join(data_dir+'/masks', idx + '.png')
        mask = io.imread(mask_name)
        label = dataset.label[i]
        
        images.append(image)
        masks.append(mask)
        labels.append(label)
        
    
    
    return images, masks, labels

In [0]:
class DermoscopyDataset(Dataset):
    """Dermoscopy dataset."""

    def __init__(self, csv_file, root_dir, transform=None, reducedSize=False):
        """
        Args:
            csv_file (string): Path al fichero csv con las anotaciones.
            root_dir (string): Directorio raíz donde encontraremos las carpetas 'images' y 'masks' .
            transform (callable, optional): Transformaciones opcionales a realizar sobre las imágenes.
        """
        dataset = pd.read_csv(csv_file,header=0,dtype={'id': str, 'label': int})
        
        

        if reducedSize:
          idx=np.random.permutation(range(len(dataset)))
          reduced_dataset=dataset.iloc[idx[0:reducedSize]]
          dataset=reduced_dataset.reset_index(drop=True)
#          
        self.imgs, self.masks, self.labels = load_data (root_dir, dataset)
#        self.root_dir = root_dir
#        self.img_dir = os.path.join(root_dir,'images') 
#        self.mask_dir = os.path.join(root_dir,'masks')
        self.transform = transform
        self.classes = ['nevus', 'melanoma', 'keratosis']
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
#        #Leemos la imagen
#        img_name = os.path.join(self.img_dir,self.dataset.id[idx] + '.jpg')
#        image = io.imread(img_name)
#        #Leemos la máscara
#        mask_name = os.path.join(self.mask_dir,self.dataset.id[idx] + '.png')
#        mask = io.imread(mask_name)
        image = self.imgs[idx]
        label = self.labels[idx]
        mask = self.masks[idx]
        
        sample = {'image': image, 'mask': mask, 'label':  label}
        if self.transform:
            sample = self.transform(sample)
        return sample

We now instantiate the class to iterate over some samples to see what we generate.


In [0]:
#train_dataset = DermoscopyDataset(csv_file='data/dermoscopyDBtrain.csv', root_dir='data')

#fig = plt.figure()

#for i in range(len(train_dataset)):
#    sample = train_dataset[i]
#    print(i, sample['image'].shape, sample['label'])

    #ax = plt.subplot(1, 4, i + 1)
    #plt.tight_layout()
    #ax.set_title('Sample #{}'.format(i))
    #ax.axis('off')
    #plt.imshow(sample['image'])

    #if i == 3:
    #    plt.show()
    #    break

In [0]:
import random
import pandas as pd
import numpy as np

# Here we subdivide the train and test set from the main dataset
# So we can validate better the parameters
# Finally we will train completelly with the whole dataset

def getTestSet(tr_set, percent):

  rows_tst = int(tr_set.shape[0]*percent)
  
  tst_set = tr_set.sample(n = rows_tst) 
  tr_set = tr_set[~tr_set['id'].isin(tst_set['id'])]

  tst_set['id'] = pd.Series([str(val).zfill(4) for val in tst_set['id']], index = tst_set.index)
  tr_set['id'] = pd.Series([str(val).zfill(4) for val in tr_set['id']], index = tr_set.index)

  print(np.intersect1d(tst_set['id'], tr_set['id']))

  return tr_set, tst_set

In [0]:
dataset = pd.read_csv('data/dermoscopyDBtrain.csv')
val_dataset = pd.read_csv('data/dermoscopyDBval.csv')
test_dataset = pd.read_csv('data/dermoscopyDBtest.csv')

#train ,test = getTestSet(dataset, 0.1)

#train.to_csv( '/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/DBtrain_1.csv', index=False)
#test.to_csv('/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/DBtest_1.csv', index=False)

#tr_path = '/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/DBtrain_1.csv'
#tst_path = '/content/drive/My Drive/University/UC3M_Master/2nd_Semester/Computer Vision/Labs/Lab6/DBtest_1.csv'

### Transforms
----------

In the previously shown examples we can see that the size of the images is not the same. This would prevent to train a red convolutional neuronal, as the vast majority require fixed-size inputs. Furthermore, the image is not always adjusted to the lesion, and indeed, in some examples lesions are very small compared to the size of the image. It would then be desirable to adjust the input images so that the lesion covers almost the entire image.

To do this, we are going to create some preprocessing code, focusing on 4 transformations:

- `` CropByMask``: to crop the image using the lesion mask
- `` Rescale``: to scale the image
- `` RandomCrop``: to crop the image randomly, it allows us to augment the data samples with random crops
- `` ToTensor``: to convert numpy matrices into torch tensors (rearranging the axes).

We will define them as callable classes instead of simple functions, as we will not need to pass the transform  parameters every time we call a method. To do this, we only have to implement the `` __call__`` method and, if necessary, the `` __init__`` method.
Then we can use a transformation with the following code:

::

    tsfm = Transform(params)
    transformed_sample = tsfm(sample)


In [0]:
class CropByMask(object):
    """Crop the image using the lesion mask.

    Args:
        border (tuple or int): Border surrounding the mask. We dilate the mask as the skin surrounding 
        the lesion is important for dermatologists.
        If it is a tuple, then it is (bordery,borderx)
    """

    def __init__(self, border):
        assert isinstance(border, (int, tuple))
        if isinstance(border, int):
            self.border = (border,border)
        else:
            self.border = border
            
    def __call__(self, sample):
        image, mask, label = sample['image'], sample['mask'],sample['label']
        h, w = image.shape[:2]
        #Calculamos los índices del bounding box para hacer el cropping
        sidx=np.nonzero(mask)
        minx=np.maximum(sidx[1].min()-self.border[1],0)
        maxx=np.minimum(sidx[1].max()+1+self.border[1],w)
        miny=np.maximum(sidx[0].min()-self.border[0],0)
        maxy=np.minimum(sidx[0].max()+1+self.border[1],h)
        #Recortamos la imagen
        image=image[miny:maxy,minx:maxx,...]
        mask=mask[miny:maxy,minx:maxx]

        return {'image': image, 'mask': mask, 'label' : label}
    
class Rescale(object):
    """Re-scale image to a predefined size.

    Args:
        output_size (tuple or int): The desired size. If it is a tuple, output is the output_size. 
        If it is an int, the smallest dimension will be the output_size
            a we will keep fixed the original aspect ratio.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, mask, label = sample['image'], sample['mask'],sample['label']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))
        msk = transform.resize(mask, (new_h, new_w))

        return {'image': img, 'mask': msk, 'label' : label}


class RandomCrop(object):
    """Randomly crop the image.

    Args:
        output_size (tuple or int): Crop size. If  int, square crop

    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, mask, label = sample['image'], sample['mask'],sample['label']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        if h>new_h:
            top = np.random.randint(0, h - new_h)
        else:
            top=0
            
        if w>new_w: 
            left = np.random.randint(0, w - new_w)
        else:
            left = 0
            
        image = image[top: top + new_h,
                     left: left + new_w]

        mask = mask[top: top + new_h,
                      left: left + new_w]


        return {'image': image, 'mask': mask, 'label': label}


class ToTensor(object):
    """Convert ndarrays into pytorch tensors."""

    def __call__(self, sample):
        image, mask, label = sample['image'], sample['mask'],sample['label']

        # Cambiamos los ejes
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image)
        # A la máscara le añadimos una dim fake al principio
        mask = torch.from_numpy(mask)
        mask = mask.unsqueeze(0)
        label=torch.tensor(label,dtype=torch.long)
        
        return {'image':image,
                'mask':mask,
                'label':label}
    
class Normalize(object):
    """Normalize data by subtracting means and dividing by standard deviations.

    Args:
        mean_vec: Vector with means. 
        std_vec: Vector with standard deviations.
    """

    def __init__(self, mean,std):
      
        assert len(mean)==len(std),'Length of mean and std vectors is not the same'
        self.mean = np.array(mean)
        self.std = np.array(std)

    def __call__(self, sample):
        image, mask, label = sample['image'], sample['mask'],sample['label']
        c, h, w = image.shape
        assert c==len(self.mean), 'Length of mean and image is not the same' 
        dtype = image.dtype
        mean = torch.as_tensor(self.mean, dtype=dtype, device=image.device)
        std = torch.as_tensor(self.std, dtype=dtype, device=image.device)
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
    
        
        return {'image': image, 'mask': mask, 'label' : label}

class RandomHorizontalFlip(object):
    """Horizontally flip the given PIL Image randomly with a given probability.

    Args:
        p (float): probability of the image being flipped. Default value is 0.5
    """

    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, sample):
        
        img = sample['image']
        mask = sample['mask']
        label = sample['label']
        
        if random.random() < self.p:
            return {'image': np.fliplr(img), 'mask': np.fliplr(mask), 'label': label}
        return sample

    def __repr__(self):
        return self.__class__.__name__ + '(p={})'.format(self.p)
    
class RandomVerticalFlip(object):
    """Horizontally flip the given PIL Image randomly with a given probability.

    Args:
        p (float): probability of the image being flipped. Default value is 0.5
    """

    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, sample):
        
        img = sample['image']
        mask = sample['mask']
        label = sample['label'] 
        
        if random.random() < self.p:
            return {'image':np.flipud(img), 'mask': np.flipud(mask), 'label': label}
        return sample

    def __repr__(self):
        return self.__class__.__name__ + '(p={})'.format(self.p)



### Composed Transforms

Now let's apply the different transformations to our images. 

We will rescale the images so that their smallest dimension is 256 and then make random crops of size 224. To compose the transformations ``Rescale`` and ``RandomCrop`` we can use ``torchvision.transforms.Compose``, which is a simple callable class.




In [0]:
#cmask = CropByMask(15)
#scale = Rescale(256)
#crop = RandomCrop(224)
#composed = transforms.Compose([CropByMask(15), Rescale(256),RandomCrop(224)])

# Apply each of the above transforms on sample.
#fig = plt.figure()
#sample = train_dataset[65]
#ax = plt.subplot(2,3, 1)
#plt.tight_layout()
#ax.set_title('original')
#plt.imshow(sample['image'])
    
#for i, tsfrm in enumerate([cmask, scale, crop, composed]):
    #transformed_sample = tsfrm(sample)

    #ax = plt.subplot(2, 3, i + 2)
    #plt.tight_layout()
    #ax.set_title(type(tsfrm).__name__)
    #plt.imshow(transformed_sample['image'])

#plt.show()

Iterating the dataset
-----------------------------

We can now put everything together to create the train and test datasets with the corresponding transformations.
In summary, every time we sample an image from the dataset (during training):
- We will read the image and the mask
- We will apply the transformations and we will crop the image using a bounding box computed from the mask
- As the final cropping operation is random, we perform data augmentation during sampling

We can easily iterate over the dataset with a ``for i in range`` loop.




In [0]:
def getDermoData(train_p, test_p):

  my_transforms = transforms.Compose([CropByMask(15),
                                    RandomVerticalFlip(),
                                    RandomHorizontalFlip(),
                                    Rescale(224),
                                    RandomCrop(224),
                                    ToTensor(),
                                    Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    ])
  #Train Dataset
  train_dataset = DermoscopyDataset(csv_file=train_p,
                                      root_dir='data',
                                      transform=my_transforms)

  #Test dataset
  test_dataset = DermoscopyDataset(csv_file=test_p,
                                      root_dir='data',
                                      transform=transforms.Compose([
                                      CropByMask(15),
                                      Rescale((224,224)),
                                      ToTensor(),
                                      Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
                                      ]))

  return train_dataset, test_dataset

In [0]:
#Train Dataset
train_dataset = DermoscopyDataset(csv_file='data/dermoscopyDBtrain.csv',
                                    root_dir='data',
                                    transform=transforms.Compose([
                                    CropByMask(15),
                                    Rescale(224),
                                    RandomCrop(224),
                                    ToTensor(),
                                    Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    ]))
#Val dataset
val_dataset = DermoscopyDataset(csv_file='data/dermoscopyDBval.csv',
                                    root_dir='data',
                                    transform=transforms.Compose([
                                    CropByMask(15),
                                    Rescale(224),
                                    RandomCrop(224),
                                    ToTensor(),
                                    Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    ]))

#Test dataset
test_dataset = DermoscopyDataset(csv_file='data/dermoscopyDBtest.csv',
                                    root_dir='data',
                                    transform=transforms.Compose([
                                    CropByMask(15),
                                    Rescale(224),
                                    RandomCrop(224),
                                    ToTensor(),
                                    Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    ]))

Finally, we have to create a dataloader allowing to:

- Sample batches of samples to feed the network during training
- Shuffle data
- Load the data in parallel using multiple cores.

``torch.utils.data.DataLoader`` is an iterator that provides all these features. An important parameter of the iterator is ``collate_fn``. We can specify how samples are organized in batches by choosing the most appropriate function. In any case, the default option should work fine in most cases.




In [0]:
def loadData(train_p, train_dataset, val_dataset, test_dataset, train_batch, testval_batch):
  #WEIGHTED SAMPLER

  db = pd.read_csv(train_p,header=0,dtype={'id': str, 'label': int})
  target = db.label.to_numpy()

  cls_weights = torch.from_numpy(compute_class_weight('balanced', np.unique(target), target))
  weights = cls_weights[target]
  sampler = WeightedRandomSampler(weights, len(target), replacement=True)

  ##Specify training dataset, with a batch size of 8, shuffle the samples, and parallelize with 4 workers
  train_dataloader = DataLoader(train_dataset, batch_size=train_batch, sampler=sampler, drop_last=True)

  #Validation dataset => No shuffle
  val_dataloader = DataLoader(val_dataset, batch_size=testval_batch, shuffle=False)

  #Test Dataset => => No shuffle
  test_dataloader = DataLoader(test_dataset, batch_size=testval_batch, shuffle=False)

  return train_dataloader, val_dataloader, test_dataloader

## Part 2: Fine-tuning a pre-trained model

In the second part of the practice we will build an automatic skin lesion diagnosis system. Instead of training a CNN designed by us from the beginning, we will fine-tune a network that has previously been trained for another task. As seen in the lectures, this usually becomes a good alternative when we do not have many data in the training dataset (in relation to the parameters to be learned).

In particular, we will use the resnet-18 CNN, included in the ``torchvision`` package.

### Performance Metric for evaluation
We will start by defining the metric we will use to evaluate our network. In particular, and following the instructions of the organizers of the original ISIC challenge, we will use the area under the ROC or AUC, but we will calculate 3 different AUCs:
- 1) AUC of binary problem melanoma vs all
- 2) AUC of the binary problem seborrheic keratosis vs all
- 3) AUC average of the previous two

The following function computes AUCs from the complete database outputs:



In [0]:
#Function that computes 2 AUCs: melanoma vs all and keratosis vs all
# scores is nx3: n is the number of samples in the dataset 
# labels is nx1
# Function resturns an array with two elements: the auc values
def computeAUCs(scores,labels):
    #0: benign nevus, 1: malignant melanoma, 2: seborrheic keratosis.            
    aucs = np.zeros((3,))
    #Calculamos el AUC benign vs all
    scores_b = scores[:,0]
    labels_b = (labels == 0).astype(np.int) 
    aucs[0]=metrics.roc_auc_score(labels_b, scores_b)
    
    #Calculamos el AUC melanoma vs all
    scores_mel = scores[:,1]
    labels_mel = (labels == 1).astype(np.int) 
    aucs[1]=metrics.roc_auc_score(labels_mel, scores_mel)

    #Calculamos el AUC queratosis vs all
    scores_sk = scores[:,2]
    labels_sk = (labels == 2).astype(np.int) 
    aucs[2]=metrics.roc_auc_score(labels_sk, scores_sk)
    
    return aucs

### Training function

We continue defining the function to train our classifier:

In [0]:
def train_model(model, criterion, optimizer, scheduler, image_datasets, dataset_sizes,
                dataloaders, device, num_epochs=25):
    since = time.time()
    
    numClasses = len(image_datasets['train'].classes)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_aucs = np.zeros((2,)) #AUCs melanoma vs all, and keratosis
    best_auc = 0
    #Loop of epochs (each iteration involves train and val datasets)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        
        
        # Cada época tiene entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set the model in training mode
            else:
                model.eval()   # Set the model in val mode (no grads)

            
            #Dataset size
            numSamples = dataset_sizes[phase]
            
            # Create variables to store outputs and labels
            outputs_m=np.zeros((numSamples,numClasses),dtype=np.float)
            labels_m=np.zeros((numSamples,),dtype=np.int)
            running_loss = 0.0
            
            contSamples=0
            
            # Iterate (loop of batches)
            for sample in dataloaders[phase]:
                inputs = sample['image'].to(device).float()
                labels = sample['label'].to(device)
#                print('labels in a batch:',labels)
                
                
                #Batch Size
                batchSize = labels.shape[0]
                
                # Set grads to zero
                optimizer.zero_grad()

                # Forward
                # Register ops only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward & parameters update only in train
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Accumulate the running loss
                running_loss += loss.item() * inputs.size(0)
                
                #Apply a softmax to the output
                outputs=F.softmax(outputs.data,dim=1)
                # Store outputs and labels 
                outputs_m [contSamples:contSamples+batchSize,...]=outputs.cpu().numpy()
                labels_m [contSamples:contSamples+batchSize]=labels.cpu().numpy()
                contSamples+=batchSize
                
            #At the end of an epoch, update the lr scheduler    
            if phase == 'train':
                scheduler.step()
            
            #Accumulated loss by epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            
            #Compute the AUCs at the end of the epoch
            aucs=computeAUCs(outputs_m,labels_m)
            
            #And the Average AUC
            epoch_auc = aucs[1:].mean()
                         
            print('{} Loss: {:.4f} AUC mel: {:.4f} sk: {:.4f} avg: {:.4f}'.format(
                phase, epoch_loss, aucs[1], aucs[2], epoch_auc))

            # Deep copy of the best model
            if phase == 'val' and epoch_auc > best_auc:
                best_auc = epoch_auc
                best_aucs = aucs.copy()        
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val AUCs: mel {:4f} sk {:4f} avg {:4f}'.format(best_aucs[1],best_aucs[2],best_auc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


Include an auxiliary function to show some predictions:

In [0]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, sample in enumerate(dataloaders['val']):
            inputs = sample['image'].to(device).float()
            labels = sample['label'].to(device)
            

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


In [0]:
def get_results(model, batchSize, dataloader, device, dataset_size, numClasses):
    
    """Evaluates the performance of the given model"""
    model.eval()
     
     #Dataset size
    numSamples = dataset_size
    
    # Create variables to store outputs and labels
    outputs_m=np.zeros((numSamples,numClasses),dtype=np.float)
    preds_m=np.zeros((numSamples,),dtype=np.int)
    #true_labels=np.zeros((numSamples,),dtype=np.int)
    contSamples=0
    #Batch Size
#    batchSize = labels.shape[0]
    with torch.no_grad():
         for samples in dataloader:
            inputs = samples['image'].to(device).float()
            #labels = samples['label'].to(device)
            
    
            outputs = model(inputs)
            outputs=F.softmax(outputs.data,dim=1)
            _, preds = torch.max(outputs, 1)
#            print('%d out of %d at batch %d'%(correct,total,i))
            
            outputs_m [contSamples:contSamples+batchSize,...]=outputs.cpu().numpy()
            preds_m [contSamples:contSamples+batchSize]=preds.cpu().numpy()
            #true_labels [contSamples:contSamples+batchSize]=labels.cpu().numpy()
            contSamples+=batchSize
    
    return preds_m, outputs_m #, true_labels
        

### Fine-tuning of a pre-trained CNN
Once we have defined the training and evaluation functions, we will fine-tune the resnet-18 CNN using our database. In addition, we define the loss, the optimizer and the lr scheduler:

In [0]:
#RESNET 18

#model_ft = models.resnet18(pretrained=True)

#num_ftrs = model_ft.fc.in_features

# We need to set-up the output layer (fully connected) to provide 3 scores (nevus, melanoma, y queratosis).
#model_ft.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

#COnvert netowrk to GPU if available
#model_ft = model_ft.to(device)

#The loss is a cross-entropy loss
#criterion = nn.CrossEntropyLoss()

# We will use SGD with momentum as optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, momentum=0.9)

# Our scheduler starts with an lr=1e-3 and decreases by a factor of 0.1 every 7 epochs.
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [0]:
#RESNET 152

#model_ft = models.resnet152(pretrained=True)

#num_ftrs = model_ft.fc.in_features

# We need to set-up the output layer (fully connected) to provide 3 scores (nevus, melanoma, y queratosis).
#model_ft.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

#COnvert netowrk to GPU if available
#model_ft = model_ft.to(device)

#The loss is a cross-entropy loss
#criterion = nn.CrossEntropyLoss()

# We will use SGD with momentum as optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, momentum=0.9)

# Our scheduler starts with an lr=1e-3 and decreases by a factor of 0.1 every 7 epochs.
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Set-up the data loaders
No we will assign the dataloaders over the training and validation data

In [0]:
#image_datasets = {'train' : train_dataset, 'val': val_dataset}

#dataloaders = {'train' : train_dataloader, 'val': val_dataloader}
          
#dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
#class_names = image_datasets['train'].classes


### Train our network

In [0]:
#model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, image_datasets,dataset_sizes, dataloaders, device, num_epochs=25)

In [0]:
#torch.save(model_ft.state_dict(), 'resnet152_1.pth')
#files.download('resnet152_1.pth')

In [0]:
#pred_labels, scores, true_labels =  get_results(model_ft, test_batch, test_dataloader, 
#                                               device, len(test_dataset), len(class_names))

#aucs = computeAUCs(scores, true_labels)


In [0]:
#print('AUC %s: %.4f'%(class_names[1],aucs[1]))
#print('AUC %s: %.4f'%(class_names[2],aucs[2]))

#print('AUC average', np.mean(aucs[1:]))

In [0]:
tstval_batch = 256

tr_batch_sizes = [8]
poss_lr = [1e-3]

In [0]:
#tr_dataset, tst_dataset = getDermoData(tr_path, tst_path)

In [26]:
for i, tr_batch in enumerate(tr_batch_sizes):

  print('Trainning batch size: ', tr_batch)

  db = pd.read_csv('data/dermoscopyDBtrain.csv',header=0,dtype={'id': str, 'label': int})
  target = db.label.to_numpy()
  cls_weights = torch.from_numpy(compute_class_weight('balanced', np.unique(target), target))
  weights = cls_weights[target]
  sampler = WeightedRandomSampler(weights, len(target), replacement=True)
  
  ##Specify training dataset, with a batch size of 8, shuffle the samples, and parallelize with 4 workers
  train_dataloader = DataLoader(train_dataset, batch_size=tr_batch, sampler=sampler, drop_last=True)

  #Validation dataset => No shuffle
  val_dataloader = DataLoader(val_dataset, batch_size=tstval_batch, shuffle=False)

  #Test Dataset => => No shuffle
  test_dataloader = DataLoader(test_dataset, batch_size=tstval_batch, shuffle=False)

  #tr_dataloader, val_dataloader, tst_dataloader = loadData(tr_path, tr_dataset, val_dataset, tst_dataset, tr_batch, tstval_batch)

  image_datasets = {'train' : train_dataset, 'val': val_dataset}
  dataloaders = {'train' : train_dataloader, 'val': val_dataloader}            
  dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

  #model_ft = models.resnet152(pretrained=True)
  model_ft = models.googlenet(pretrained=True)
  model_ft.fc = nn.Linear(model_ft.fc.in_features, len(train_dataset.classes))
  model_ft = model_ft.to(device)
  criterion = nn.CrossEntropyLoss()

  for j, learning_rate in enumerate(poss_lr):

    print('Learning rate: ', learning_rate)
    
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    class_names = image_datasets['train'].classes

    model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, image_datasets,dataset_sizes, dataloaders, device, num_epochs=50)

    _, scores =  get_results(model_ft, tstval_batch, test_dataloader, device, len(test_dataset), len(class_names))

    #aucs = computeAUCs(scores, true_labels)

    #print('AUC %s: %.4f'%(class_names[1],aucs[1]))
    #print('AUC %s: %.4f'%(class_names[2],aucs[2]))
    #print('AUC average', np.mean(aucs[1:]))

    #print('------------------------------------------------------------------------')

    model_name = 'googleNet_batch_'+str(tr_batch)+ '_lr_' +str(learning_rate)+'.pth'
    csv_name = 'googleNet_batch_'+str(tr_batch)+ '_lr_' +str(learning_rate)+'.csv'
    np.savetxt(csv_name, scores, delimiter=",")
    torch.save(model_ft.state_dict(), model_name)

Trainning batch size:  8


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth



Learning rate:  0.001
Epoch 0/49
----------
train Loss: 0.8757 AUC mel: 0.7432 sk: 0.8379 avg: 0.7906
val Loss: 0.6754 AUC mel: 0.7767 sk: 0.9191 avg: 0.8479

Epoch 1/49
----------
train Loss: 0.6626 AUC mel: 0.8583 sk: 0.9358 avg: 0.8971
val Loss: 0.6684 AUC mel: 0.7953 sk: 0.9312 avg: 0.8632

Epoch 2/49
----------
train Loss: 0.5882 AUC mel: 0.8920 sk: 0.9490 avg: 0.9205
val Loss: 0.6028 AUC mel: 0.8147 sk: 0.9495 avg: 0.8821

Epoch 3/49
----------
train Loss: 0.4852 AUC mel: 0.9219 sk: 0.9757 avg: 0.9488
val Loss: 0.6526 AUC mel: 0.8522 sk: 0.9162 avg: 0.8842

Epoch 4/49
----------
train Loss: 0.4815 AUC mel: 0.9207 sk: 0.9743 avg: 0.9475
val Loss: 0.7284 AUC mel: 0.7856 sk: 0.9458 avg: 0.8657

Epoch 5/49
----------
train Loss: 0.3827 AUC mel: 0.9492 sk: 0.9875 avg: 0.9683
val Loss: 0.7621 AUC mel: 0.7589 sk: 0.9129 avg: 0.8359

Epoch 6/49
----------
train Loss: 0.3853 AUC mel: 0.9554 sk: 0.9812 avg: 0.9683
val Loss: 0.6713 AUC mel: 0.8075 sk: 0.9597 avg: 0.8836

Epoch 7/49
-------

## Part 3: Evaluation (Important)
The evaluation of this practice will be done through a challenge. For this, students are asked the following to provide an output matrix for both the validation and test database using this code. The matrix will have a size 600x3, 600 lesions and the 3 classes considered in the problem. The matrix must be provided in csv format (with 3 numbers per row separated by ',').

In addition, students will submit a short report (1 side for the description, 1 side for references and figures if necessary) where they will describe the most important aspects of the proposed solution and include a table with the validation results achieved by their extensions/decisions. The objective of this report is for the teacher to assess the developments / extensions / decisions made by the students when optimizing their system. And the table is asked to demonstrate that, at least in validation, their decisions helped to improve the system performance.  You don't need to provide an absolute level of detail about the changes made, just list them, briefly discuss their purpose and show their impact in the table.

The deadline for delivery of the results file and the report is Monday May 11 at 22:00.

